In [14]:
import geemap
import ee
ee.Initialize()
import scripts.analysis_functions as af
import scripts.fire_info as fi
import scripts.get_image_collections as gic
import scripts.get_image_collections2 as gic2
import scripts.utils as utils
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# For yearly fires (takes a FeatureCollection)

In [17]:
def bs_calc(feat: ee.Feature):
    fire = ee.Feature(feat)
    fire_geom = fire.geometry()
    # pass one fire feature into new fire_info function
    region,pre_start,pre_end,fire_start,fire_end = fi.get_fire_info_from_feature(fire, run_mode)

    # key consideration for the recent mode is how to handle more recent fires that may have cloudy imagery... 
    # can put date filter value in gic.remove_recent() further back from today's date (currently its 30 days) so all fires left in the FeatureColl should have enough images to 
    # get a good .mean() composite using gic.make_pre_composite(), or keep the date filter pretty close to today's date and let it pull T1 realtime and toa imgs with no cloudBusting (as is done currently) with gic.make_nrt_composite()
    # and just caveat that the more recent fire perims will have less reliable BS metric; its easy to run this every few months to get a better severity estimate on those more recent fires
    sensor = "landsat"
    pre_collection = gic2.getLandsatToa(pre_start,pre_end,fire_geom)
    #pre_collection = gic.get_image_collection(sensor,region,pre_start,pre_end,landsatCol='bestls',cloudBustingMethod='bust')
    pre_img = gic.get_composite(pre_collection,gic.make_pre_composite,pre_start,pre_end)

    if run_mode == 'historical':
        post_collection = gic2.getLandsatToa(fire_start,fire_end,fire_geom)
        # post_collection = gic.get_image_collection(sensor,region,fire_start,fire_end,landsatCol='bestls', cloudBustingMethod='bust')
        post_img = gic.get_composite(post_collection,gic.make_pre_composite,fire_start, fire_end) 

    else: # recent
        post_collection = gic2.getLandsatToa(fire_start,fire_end,fire_geom)
        # post_collection = gic.get_image_collection(sensor,region,fire_start,fire_end)
        post_img = gic.get_composite(post_collection,gic.make_nrt_composite,sensor) 
    
    rdnbr = af.rdnbr(pre_img,post_img)
    miller = af.miller_thresholds(rdnbr)
    
    return ee.Image(miller).clip(region).select('MillersThresholds').toByte()

## NIFC annual perims 2021 & 2022 to supplement MTBS annual fire perims 2013-2020

In [20]:
# One set of yearly fires, no need for remove_recent() or year/ac filtering, that was already done upon data fetch
# NIFC
yr='2021'
fires = ee.FeatureCollection("projects/pyregence-ee/assets/conus/nifc/nifc_fires_{yr}_gte500acres_20221004")
print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')

# filter 2022 fires for a July 1 cut-off
# fires = fires.filter(ee.Filter.lte('Discovery','2022-07-01'))
# print(fires.sort('Discovery',False).first().getInfo()['properties'])

latest_fire = fires.sort('Discovery',False).first()
print('latest fire: ', latest_fire.get('Discovery').getInfo())

run_mode = fi.config_mode(latest_fire)
print(f'run mode: {run_mode}')

bs_imgColl = ee.FeatureCollection(fires).map(bs_calc)
annual_conus_bs = ee.ImageCollection(bs_imgColl).max().add(1).rename('SEVERITY') #shift severity values +1 so 0 can be nodata
# To Asset
desc = f'nifc_bs_{yr}_gte500acres_20221004'
utils.exportImgtoAsset(annual_conus_bs, 
                    desc=desc,
                    region=None,
                    asset_folder= "projects/pyregence-ee/assets/conus/nifc", 
                    export_type='conus',
                    export=True)
                    #default=sensor)

Total Fires in FeatureCollection: 589
latest fire:  2021-12-30
run mode: historical
export task started: projects/pyregence-ee/assets/conus/nifc/nifc_bs_2021_gte500acres_20221004


## MTBS annual fire perims 2013-2020

In [28]:
# Running for each year of MTBS fire perims 
fires = ee.FeatureCollection("projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_all_20220928")
print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')
acre_min = 0 # can filter by Acres field 
years = [2013,2014,2015,2016,2017,2018,2019,2020,2021]

for y in years:
    fires_w_filters = gic.remove_recent(fires).filter(ee.Filter.And(ee.Filter.gte('BurnBndAc', acre_min),ee.Filter.eq('Year',y)) )
    print(f'Removing very recent fires... filtering fires above {acre_min} acres that burned in {y}')
    print(f'Computing Burn Severity for {fires_w_filters.size().getInfo()} fires')

    latest_fire = fires_w_filters.sort('Discovery',False).first()
    print('latest fire: ', latest_fire.get('Discovery').getInfo())

    run_mode = fi.config_mode(latest_fire)
    print(f'run mode: {run_mode}')
    
    bs_imgColl = ee.FeatureCollection(fires_w_filters).map(bs_calc)
    annual_conus_bs = ee.ImageCollection(bs_imgColl).max().add(1).rename('SEVERITY').set('Year',y) #shift severity values +1 so 0 can be nodata
    
    # To Asset
    desc = f'mtbs_annual_bs_{str(y)}'
    utils.exportImgtoAsset(annual_conus_bs, 
                        desc=desc,
                        region=None,
                        asset_folder= "projects/pyregence-ee/assets/conus/mtbs", 
                        export_type='conus',
                        export=True)
                        #default=sensor)
    print('\n')

Removing very recent fires... filtering fires above 0 acres that burned in 2013
Computing Burn Severity for 672 fires
latest fire:  2013-12-19
run mode: historical
export task started: projects/pyregence-ee/assets/conus/mtbs/mtbs_annual_bs_2013


Removing very recent fires... filtering fires above 0 acres that burned in 2014
Computing Burn Severity for 1000 fires
latest fire:  2014-12-16
run mode: historical
export task started: projects/pyregence-ee/assets/conus/mtbs/mtbs_annual_bs_2014


Removing very recent fires... filtering fires above 0 acres that burned in 2015
Computing Burn Severity for 1151 fires
latest fire:  2015-12-26
run mode: historical
export task started: projects/pyregence-ee/assets/conus/mtbs/mtbs_annual_bs_2015


Removing very recent fires... filtering fires above 0 acres that burned in 2016
Computing Burn Severity for 1331 fires
latest fire:  2016-12-30
run mode: historical
export task started: projects/pyregence-ee/assets/conus/mtbs/mtbs_annual_bs_2016


Removing 

In [29]:
Map = geemap.Map()
Map.addLayer(annual_conus_bs, {"min":1,"max":4,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'bs img')

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# For one fire (takes a Feature)
### Select a fire from a pre-existing fire featurecollection or provide your own fire feature asset (must have 'Discovery' property with value of format ee.String('yyyy-mm-dd') )


In [ ]:
# NIFC
fires = ee.FeatureCollection("projects/pyregence-ee/assets/conus/nifc/nifc_fires_2021_gte500acres_20221004")
# fires = ee.FeatureCollection("projects/pyregence-ee/assets/conus/nifc/nifc_fires_2022_gte500acres_20221004")
# #fire = fires_w_filters.filter(ee.Filter.eq('Name', 'Dixie')).sort('Acres',False).first()
fire = fires.sort('Discovery',False).first()
fire_geom = ee.Feature(fire).geometry()

run_mode = fi.config_mode(fire)
print(f'run mode: {run_mode}')

print(fire.getInfo()['properties'])

# pass one fire feature into new fire_info function
region,pre_start,pre_end,fire_start,fire_end = fi.get_fire_info_from_feature(fire, run_mode)
print(pre_start.getInfo())
print(pre_end.getInfo())
print(fire_start.getInfo())
print(fire_end.getInfo())

sensor = "landsat"
pre_collection = gic2.getLandsatToa(pre_start,pre_end,fire_geom)
#pre_collection = gic2.get_image_collection(sensor,region,pre_start,pre_end,landsatCol='bestls',cloudBustingMethod='bust')
pre_img = gic.get_composite(pre_collection,gic.make_pre_composite,pre_start,pre_end)

if run_mode == 'historical':
    post_collection = gic2.getLandsatToa(fire_start,fire_end,fire_geom)
    # post_collection = gic.get_image_collection(sensor,region,fire_start,fire_end,landsatCol='bestls', cloudBustingMethod='bust')
    post_img = gic.get_composite(post_collection,gic.make_pre_composite,fire_start, fire_end) 

else: # recent
    post_collection = gic2.getLandsatToa(fire_start,fire_end,fire_geom)
    # post_collection = gic.get_image_collection(sensor,region,fire_start,fire_end)
    post_img = gic.get_composite(post_collection,gic.make_nrt_composite, sensor) 

print(pre_collection.size().getInfo())
print(pre_img.bandNames().getInfo())

print(post_collection.size().getInfo())
print(post_img.bandNames().getInfo())

rdnbr = af.rdnbr(pre_img,post_img)
miller = af.miller_thresholds(rdnbr).add(1).clip(region).select('MillersThresholds').toByte()

In [ ]:
# look at post_img and determine for yourself if its good enough to be using for analysis 
Map = geemap.Map()
Map.addLayer(pre_img,{"bands":["SWIR2","NIR","RED"],"min":0,"max":0.848},"pre")
Map.addLayer(post_img,{"bands":["SWIR2","NIR","RED"],"min":0,"max":0.848},"post")
Map.addLayer(rdnbr.clip(region),{"min":0,"max":1500,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'rdnbr')
Map.addLayer(miller.clip(region),{"min":1,"max":4,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'miller')
Map.addLayer(region, {}, 'region')
Map.centerObject(region, 11)
Map



## Exports
### To export an annual set of fires, use export_type = 'conus', if its a single fire, use export_type = 'single_fire'

In [ ]:
CA = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME','California')).first()
# Map.addLayer(ee.Feature(CA))
# print(ee.Feature(CA).getInfo()['properties'])
                                                      
# Single FIre export templates
# # To Google Drive
# desc = 'dixie_bs2020_test_20211227'
# utils.exportImgtoDrive(miller,
#                         desc,
#                         export_type='single_fire',
#                         folder='BurnSeverity_outputs',
#                         export=True,
#                         default=sensor)


# # To Asset
# desc = 'dixie_bs2020_test_20211227'
# utils.exportImgtoAsset(miller, 
#                         desc=desc, 
#                         asset_folder= "users/kwoodward/burn-severity/bs_images", 
#                         export_type='single_fire',
#                         export=True,
#                         default=sensor)

# # CONUS/multi-fire export template
# # To Google Drive
# desc = 'CA_bs2021_nifcFires_20220713'
# utils.exportImgtoDrive(annual_conus_bs.clip(CA),
#                         desc,
#                         export_type='conus',
#                         folder='BurnSeverity_outputs',
#                         export=False)
#                         #default=sensor)


# To Asset
desc = 'CA_bs2021_nifcFires_20220713'
utils.exportImgtoAsset(annual_conus_bs.clip(CA), 
                        desc=desc, 
                        asset_folder= "users/kwoodward/burn-severity/bs_images", 
                        export_type='conus',
                        export=True)
                        #default=sensor)